## Data Extraction

In [16]:
import zipfile
import pandas as pd
import numpy as np

In [17]:
path_to_zip_file = './Data/archive.zip'

In [20]:
def file_extract_from_zip(zip_path):
    
    csv_file_list = []
    json_file_list = []
    
    with zipfile.ZipFile(zip_path, mode="r") as archive:
        for filename in archive.namelist():
            file_extension = filename[filename.index('.')+1:]
            if file_extension == 'csv':
                df = pd.read_csv(archive.open(filename), encoding='latin-1')
                df['region'] = filename[0:2]
                csv_file_list.append(df)
            elif file_extension == 'json':
                df_region = pd.read_json(archive.open(filename))
                df_id = pd.read_json((df_region)['items'].to_json(), orient='index')['id']
                df_category_title = pd.read_json(pd.read_json(df_region['items'].to_json(), orient='index')['snippet'].to_json(), orient='index')['title']
                data = {'category_id' : df_id, 'category_title': df_category_title}
                df = pd.concat(data, axis=1)
                df['region'] = filename[0:2]
                json_file_list.append(df)
    return pd.concat(csv_file_list, ignore_index=True), pd.concat(json_file_list, ignore_index=True)

In [21]:
df_all_region, df_all_category = file_extract_from_zip(path_to_zip_file)

## Combining Data

In [5]:
combined_df = pd.merge(df_all_region, df_all_category, on=['region', 'category_id'], how='left')

In [6]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375942 entries, 0 to 375941
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                375942 non-null  object
 1   trending_date           375942 non-null  object
 2   title                   375942 non-null  object
 3   channel_title           375942 non-null  object
 4   category_id             375942 non-null  int64 
 5   publish_time            375942 non-null  object
 6   tags                    375942 non-null  object
 7   views                   375942 non-null  int64 
 8   likes                   375942 non-null  int64 
 9   dislikes                375942 non-null  int64 
 10  comment_count           375942 non-null  int64 
 11  thumbnail_link          375942 non-null  object
 12  comments_disabled       375942 non-null  bool  
 13  ratings_disabled        375942 non-null  bool  
 14  video_error_or_removed  375942 non-n

In [7]:
combined_df[pd.isnull(combined_df['category_title'])]['category_id'].value_counts()

29    2738
Name: category_id, dtype: int64

In [8]:
df_all_category[df_all_category['category_id'] == 29]

,category_id,category_title,region
295,29,Nonprofits & Activism,US


In [9]:
sum(df_all_region[(df_all_region['category_id'] == 29) & (df_all_region['region'] != 'US')]['region'].value_counts())

2738

In [10]:
combined_df_without_null = combined_df[~pd.isnull(combined_df['category_title'])]

In [11]:
combined_df_without_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373204 entries, 0 to 375941
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                373204 non-null  object
 1   trending_date           373204 non-null  object
 2   title                   373204 non-null  object
 3   channel_title           373204 non-null  object
 4   category_id             373204 non-null  int64 
 5   publish_time            373204 non-null  object
 6   tags                    373204 non-null  object
 7   views                   373204 non-null  int64 
 8   likes                   373204 non-null  int64 
 9   dislikes                373204 non-null  int64 
 10  comment_count           373204 non-null  int64 
 11  thumbnail_link          373204 non-null  object
 12  comments_disabled       373204 non-null  bool  
 13  ratings_disabled        373204 non-null  bool  
 14  video_error_or_removed  373204 non-n

## Data Cleaning and Transformation

In [13]:
columns_to_drop = ['title', 'tags', 'thumbnail_link', 'description']

In [14]:
combined_df_clean = combined_df_without_null.drop(columns=columns_to_drop)

In [15]:
combined_df_clean.to_csv('./Data/Combined_DF.csv', index=False)